In [3]:
# INSTALL ULTRALYTICS
!pip install -U ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 67.9 MB/s eta 0:00:00


In [4]:
# IMPORT STATEMENTS
import os
import random
import shutil
from ultralytics import YOLO
import numpy as np
import matplotlib.pyplot as plt
import torch
from pathlib import Path

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [13]:
# MOUNT GOOGLE DRIVE
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!cp -r "/content/drive/MyDrive/Thesis/Object_Identification/data_cows_cls/" "/content/data_cows_cls/"
DATASET = Path("/content/data_cows_cls")
assert DATASET.exists(), f"DATASET not found: {DATASET}"
print("Using dataset:", DATASET)

Mounted at /content/drive
Using dataset: /content/data_cows_cls


In [14]:
# CHECK IMAGE COUNTS
IMAGE_EXTS = {".jpg",".jpeg",".png",".bmp",".tif",".tiff",".webp"}

def count_images(root: Path):
    counts = {}
    for split in ("train","val","test"):
        d = root / split
        if not d.exists():
            print("[warn] missing:", d);
            continue
        for cls in sorted([p for p in d.iterdir() if p.is_dir()]):
            n = sum(1 for p in cls.iterdir() if p.suffix.lower() in IMAGE_EXTS)
            counts[(split, cls.name)] = n
    return counts

for (split, cls), n in count_images(DATASET).items():
    print(f"{split:5s} | {cls:20s} : {n} images")

train | Bella                : 248 images
train | Buttercup            : 200 images
train | Daisy                : 274 images
train | Marigold             : 318 images
train | Neighbor             : 20 images
train | Rosie                : 262 images
val   | Bella                : 53 images
val   | Buttercup            : 43 images
val   | Daisy                : 59 images
val   | Marigold             : 68 images
val   | Neighbor             : 4 images
val   | Rosie                : 56 images
test  | Bella                : 54 images
test  | Buttercup            : 43 images
test  | Daisy                : 59 images
test  | Marigold             : 68 images
test  | Neighbor             : 4 images
test  | Rosie                : 56 images


In [15]:
# RUN YOLO TRAINING
LOCAL_PROJECT = Path("/content/YOLO_CowID_Runs")
LOCAL_PROJECT.mkdir(parents=True, exist_ok=True)
RUN_NAME = "y11cls_224"

model = YOLO("yolo11x-cls.pt")
model.train(
    data=str(DATASET),
    epochs=120,
    imgsz=224,
    batch=64,
    lr0=0.01,
    patience=15,
    device=0,
    workers=8,
    project=str(LOCAL_PROJECT),
    name=RUN_NAME,
    exist_ok=True
)

# Sync artifacts to Drive afterwards
DRIVE_PROJECT = Path("/content/drive/MyDrive/Thesis/Object_Identification/YOLO_CowID_Runs")
DRIVE_PROJECT.mkdir(parents=True, exist_ok=True)
!rsync -a --info=progress2 "{LOCAL_PROJECT}/" "{DRIVE_PROJECT}/"

Ultralytics 8.3.189 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data_cows_cls, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=120, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11x-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=y11cls_224, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=15, perspective=0.0, plots=True, pose=12.0